### select a chat model
OpenAi - gpt-4o-mini

In [ ]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv(override=True)

model = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv("METIS_API_KEY"),
    base_url=os.getenv("METIS_BASE_URL"),
)

### select a embedding model
OpenAi - text-embedding-3-large

In [3]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    api_key=os.getenv("METIS_API_KEY"),
    base_url=os.getenv("METIS_BASE_URL"),
)

### select a vector store
chroma

In [4]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="question",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

### extract files in lessons

In [6]:
from langchain_core.documents import Document

def load_lessons() -> list[Document]:
    """
    خواندن تمام فایل‌های درسی از پوشه lessons
    
    Returns:
        list[Document]: لیست اسناد LangChain
    """
    lessons_dir = "./lessons"
    
    if not os.path.exists(lessons_dir):
        print("❌ پوشه lessons پیدا نشد!")
        return []
    
    lesson_files = [f for f in os.listdir(lessons_dir) if f.endswith(".txt")]
    
    if not lesson_files:
        print("❌ فایل txt در پوشه lessons پیدا نشد!")
        return []
    
    documents = []
    
    for lesson_file in lesson_files:
        lesson_path = os.path.join(lessons_dir, lesson_file)
        lesson_name = os.path.splitext(lesson_file)[0]
        
        print(f"📘 در حال خواندن {lesson_name}...")
        
        with open(lesson_path, "r", encoding="utf-8") as f:
            content = f.read()
        
        # ساخت Document با metadata
        doc = Document(
            page_content=content,
            metadata={
                "source": lesson_name,
                "file_path": lesson_path
            }
        )
        
        documents.append(doc)
        print(f"   ✅ {len(content)} کاراکتر خوانده شد")
    
    print(f"\n🎉 {len(documents)} فایل با موفقیت خوانده شد ✅\n")
    
    return documents

### chunking with splitter

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(load_lessons())

print(len(all_splits))

📘 در حال خواندن lesson_01...
   ✅ 2502 کاراکتر خوانده شد
📘 در حال خواندن lesson_02...
   ✅ 4236 کاراکتر خوانده شد
📘 در حال خواندن lesson_03...
   ✅ 3661 کاراکتر خوانده شد
📘 در حال خواندن lesson_04...
   ✅ 4378 کاراکتر خوانده شد

🎉 4 فایل با موفقیت خوانده شد ✅

27


### add documents to db

In [14]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

['2f4c94ee-4ae4-4f49-a3bb-59bba8c0c4de', '177312fa-6848-4975-8204-6c6f31d824e3', '97d48243-120e-4cbb-88c8-7aad0ae52771']


### use middleware for retrivel

In [15]:
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message


agent = create_agent(model, tools=[], middleware=[prompt_with_context])

### test app

In [17]:
query = (
    "درست، نادرست در کتابخانه کوچوک ما یعنی درس اول  رو برام به صورت کامل بنویس"
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

درست، نادرست در کتابخانه کوچوک ما یعنی درس اول  رو برام به صورت کامل بنویس
================================== Ai Message ==================================

درست و نادرست (درس اول: کتاب‌خانه‌ی کلاس ما)

١. بچّه‌ها در گروه گفت‌وگو کردند و پیشنهاد خود را روی تخته‌ی کلاس نوشتند.  
   نادرست

٢. بهتر است کتاب‌ها و مجلّه‌های ویژه خودمان را بخوانیم.  
   درست

٣. همه‌ی نوشته‌های موجود در رایانه برای ما مُفید است.  
   نادرست

٤. آموزگار از دانش‌آموزان تشکّر کرد و به آن‌ها گفت که کتاب‌های خوب می‌توانند به ما کمک کنند.  
   درست

لطفاً در صورت نیاز به توضیحات بیشتر یا سوالات دیگر، بپرسید!
